# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_1}-\bar{X_2}$$

where group1 is SCN Expression values and group2 are Whole Brain Expression values 
(SCN mean - WB mean) **(Logged values, so minus gives ratio)**

$$  {Pooled\ Standard\  Deviantion} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [4]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations


In [5]:
prefix = '430AV2_'   # define a prefix to add to column names (making indexing easier later)

In [6]:
df=pd.read_table('../AltAnalyze_output/DATASET-430aV2.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(22690, 32)

In [7]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(19706, 32)

## Look at column names and then setup filters for grouping columns into SCN and WB groups

In [8]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM159292.CEL', 'GSM159293.CEL',
       'GSM159294.CEL', 'avg-SCN', 'log_fold-SCN_vs_WB', 'fold-SCN_vs_WB',
       'rawp-SCN_vs_WB', 'adjp-SCN_vs_WB', 'GSM189596.CEL', 'GSM189598.CEL',
       'GSM189600.CEL', 'GSM189602.CEL', 'GSM189628.CEL', 'GSM189630.CEL',
       'GSM189632.CEL', 'GSM189634.CEL', 'avg-WB', 'ANOVA-rawp', 'ANOVA-adjp',
       'largest fold'],
      dtype='object')

In [9]:
# define regular expressions for filters 
scn_filt ='159'
wb_filt ='189'

In [10]:
df_scn=df.filter(regex= scn_filt)
df_scn.shape

(19706, 3)

In [11]:
df_wb=df.filter(regex= wb_filt)
df_wb.shape

(19706, 8)

## Calculations 

In [12]:
SCNcount = df.filter(regex=scn_filt).count(axis=1)

In [13]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=scn_filt).mean(axis=1) - df.filter(regex=wb_filt).mean(axis=1)

In [14]:
df[prefix+'Enrich'].head()

Probesets
1427138_at      0.564749
1425600_a_at   -0.880608
1450135_at      0.743188
1434334_at      0.043725
1435780_at     -2.279606
Name: 430AV2_Enrich, dtype: float64

In [15]:
# Pooled StDev
SCNcount = df.filter(regex=scn_filt).count(axis=1)
WBcount = df.filter(regex=wb_filt).count(axis=1)

StdevSCN = (SCNcount-1) * df.filter(regex=scn_filt).var(axis=1)
StdevWB = (WBcount-1) * df.filter(regex=wb_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevSCN+StdevWB)/(SCNcount+ WBcount-2))

In [16]:
#Cohen's d

df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [17]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at       2.856382
1425600_a_at    -3.731278
1450135_at       1.893515
1434334_at       0.129605
1435780_at     -10.571958
Name: 430AV2_Cohens_d, dtype: float64

In [18]:
#J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(SCNcount+WBcount-1)))                              


In [19]:
#Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [20]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at      2.642153
1425600_a_at   -3.451432
1450135_at      1.751501
1434334_at      0.119885
1435780_at     -9.779061
Name: 430AV2_Hedges_g, dtype: float64

In [21]:
#Var_d
SCNcount = df.filter(regex=scn_filt).count(axis=1)
WBcount = df.filter(regex=wb_filt).count(axis=1)

Ftop1 = SCNcount + WBcount
Ftop2 = SCNcount * WBcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(SCNcount + WBcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [22]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.829193
1425600_a_at    1.091171
1450135_at      0.621306
1434334_at      0.459097
1435780_at      5.538620
Name: 430AV2_Var_d, dtype: float64

In [23]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [24]:
#SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [25]:
df.sort_values(by='430AV2_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_Enrich,430AV2_poolStDev,430AV2_Cohens_d,430AV2_J,430AV2_Hedges_g,430AV2_Var_d,430AV2_Var_g,430AV2_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1419408_at,Six6,sine oculis-related homeobox 6 homolog (Drosop...,ENSMUSG00000021099,20476,NaN,multicellular organismal development // regula...,transcription factor activity // DNA binding /...,nucleus,NaN,"mmu-miR-103(miRanda), mmu-miR-107(miRanda), mm...",...,1.073934e-10,5.685188,5.685188,0.083330,68.224627,0.925,63.107780,212.031048,181.419065,13.469189
1427198_at,BC022960,cDNA sequence BC022960,ENSMUSG00000081137|ENSMUSG00000050528,237246,NaN,NaN,NaN,NaN,NaN,NaN,...,1.117228e-09,3.452502,3.452502,0.080972,42.638334,0.925,39.440459,83.095947,71.098970,8.432020
1417765_a_at,Amy1,"amylase 1, salivary",ENSMUSG00000074264,11722,NaN,metabolic process // carbohydrate metabolic pr...,"calcium ion binding // hydrolase activity, act...",extracellular region // extracellular space,NaN,NaN,...,1.568185e-09,5.717668,5.717668,0.145965,39.171594,0.925,36.233725,70.204415,60.068652,7.750397
1429982_at,Rimklb,RIKEN cDNA 4933426K21 gene,ENSMUSG00000040649,108653,NaN,protein modification process,manganese ion binding // catalytic activity //...,NaN,NaN,"mmu-let-7d(miRanda), mmu-let-7f(miRanda), mmu-...",...,1.568185e-09,3.929703,3.929703,0.101171,38.842147,0.925,35.928986,69.036170,59.069073,7.685641
1436172_at,9530028C05,"predicted gene, 20559",NaN,330256,NaN,NaN,NaN,NaN,NaN,NaN,...,1.568185e-09,2.825008,2.825008,0.072868,38.769010,0.925,35.861334,68.778159,58.848312,7.671265
1417704_a_at,Arhgap6,Rho GTPase activating protein 6,ENSMUSG00000031355,11856,NaN,regulation of GTPase activity // signal transd...,Rho GTPase activator activity // GTPase activa...,cytoplasm // intracellular // actin cytoskeleton,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,1.883603e-09,3.563400,3.563400,0.097240,36.645257,0.925,33.896863,61.498100,52.619312,7.253917
1425603_at,Tmem176a,transmembrane protein 176A,NaN,66058,NaN,platelet activating factor biosynthetic process,diacylglycerol cholinephosphotransferase activity,integral to membrane // membrane,NaN,NaN,...,2.310769e-09,4.995611,4.995611,0.144743,34.513593,0.925,31.925073,54.603246,46.719902,6.835196
1455696_a_at,Prpf4b,PRP4 pre-mRNA processing factor 4 homolog B (y...,NaN,19134,NaN,protein amino acid phosphorylation // RNA spli...,transferase activity // protein kinase activit...,spliceosome // nucleus // chromosome,GenMAPP-mRNA_processing_binding_Reactome // mR...,NaN,...,3.419728e-09,5.938186,5.938186,0.183881,32.293559,0.925,29.871542,47.861695,40.951663,6.399349
1455265_a_at,Rgs16,regulator of G-protein signaling 16,ENSMUSG00000026475,19734,NaN,G-protein coupled receptor protein signaling p...,signal transducer activity // GTPase activator...,cytoplasm,GenMAPP-Calcium_regulation_in_cardiac_cells //...,"mmu-let-7(TargetScan), mmu-let-7a(miRanda|pict...",...,3.568840e-09,4.479957,4.479957,0.139976,32.005246,0.925,29.604853,47.019050,40.230675,6.342766


In [26]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM159292.CEL', 'GSM159293.CEL',
       'GSM159294.CEL', 'avg-SCN', 'log_fold-SCN_vs_WB', 'fold-SCN_vs_WB',
       'rawp-SCN_vs_WB', 'adjp-SCN_vs_WB', 'GSM189596.CEL', 'GSM189598.CEL',
       'GSM189600.CEL', 'GSM189602.CEL', 'GSM189628.CEL', 'GSM189630.CEL',
       'GSM189632.CEL', 'GSM189634.CEL', 'avg-WB', 'ANOVA-rawp', 'ANOVA-adjp',
       'largest fold', '430AV2_Enrich', '430AV2_poolStDev', '430AV2_Cohens_d',
       '430AV2_J', '430AV2_Hedges_g', '430AV2_Var_d', '430AV2_Var_g',
       '430AV2_SEg'],
      dtype='object')

### Import key file from BioMart and index probesets to MGI gene symbols

In [27]:
dfX=pd.read_table('../BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [28]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that not duplicate entries exist in the dataframe

,GSM159292.CEL,GSM159293.CEL,GSM159294.CEL,avg-SCN,log_fold-SCN_vs_WB,fold-SCN_vs_WB,rawp-SCN_vs_WB,adjp-SCN_vs_WB,GSM189596.CEL,GSM189598.CEL,...,ANOVA-adjp,largest fold,430AV2_Enrich,430AV2_poolStDev,430AV2_Cohens_d,430AV2_J,430AV2_Hedges_g,430AV2_Var_d,430AV2_Var_g,430AV2_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average SCN and WB expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [29]:
# df3 = df_FINAL1.loc[:,[u'avg-WB', u'avg-SCN', u'log_fold-SCN_vs_WB']]
# df3.columns =[prefix+'avg-WB', prefix+'avg-SCN', prefix+'log_fold-SCN_vs_WB']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [30]:
df4 = df_FINAL1.loc[:,[u'430AV2_Enrich',u'430AV2_Hedges_g', u'430AV2_Var_g', u'430AV2_SEg']]
df4.to_csv('input_files/430AV2_SymbolforIndexHedges.csv')

In [31]:
df4.head(10)  # check final ouput

,430AV2_Enrich,430AV2_Hedges_g,430AV2_Var_g,430AV2_SEg
MGI symbol,,,,
0610005C13Rik,0.071840,0.392799,0.399175,0.631803
0610008F07Rik,-0.034139,-0.240750,0.394796,0.628328
0610009B22Rik,0.951853,5.058482,1.555264,1.247102
0610009D07Rik,0.011381,0.502373,0.895966,0.943195
0610009O20Rik,-0.916283,-5.843401,1.944222,1.394354
0610010K14Rik,-0.678961,-1.304555,0.469519,0.685215
0610012G03Rik,-0.344126,-1.029226,0.675020,0.798407
0610031J06Rik,-1.822928,-3.522490,0.956158,0.977834
0610037L13Rik,1.201002,4.925677,1.494993,1.222699
